<div dir="rtl">
<center>
<span style="color:red"><h1> به نام خدا</h1></span>
<h2>بخش Elasticsearch پروژه پایانی</h2>
<span style="color:blue"><h3>سامانه بازیابی اطلاعات برای آیه های قرآنی</h3></span>
</center>

</div>


<div dir="rtl">
در این notebook، تصمیم داریم تا مراحلی که برای index کردن و تمیز کردن داده های قرآنی در elasticsearch طی کردیم را به صورت مستند داشته باشیم.<br/>
مراحلی که طی شد به طور کلی به صورت زیر است:
<ol>
<li>نصب و راه اندازی elasticSearch</li>
<li>تبدیل داده های قرآنی به Dataframe و سپس تبدیل آن به داده های مناسب برای elasticsearch</li>
<li>اعمال settingها و mapping های مورد نظر برای تمیز کردن داده های قرآنی</li>
<li>آپلود تمامی داده های قرآنی به دیتابیس elasticsearch</li>
<li>اعمال پرسمان</li>
</ol>
در نهایت، ۱۰ پرسمان که در بخش های دیگر خروجی هایشان را بررسی کردیم را روی elasticsearch نیز اعمال میکنیم و RR هر کوئری و در نهایت MRR سیستم را محاسبه میکنیم.
</div>


<div dir="rtl">
<span style="color:red"><h3>۱- نصب و راه اندازی elasticsearch</h3></span>
</div>

<div dir="rtl">
در این بخش، ابتدا از سایت elasticsearch برنامه را دانلود میکنیم و سپس آن را روی پورت 9200 اجرا میکنیم.</br>
حال از روی این پورت میتوانیم تا با elasticsearch ارتباط داشته باشیم.
</div>

In [3]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import elasticsearch
import pandas as pd
import warnings

<div dir="rtl">
ابتدا elasticsearch را روی سیستم روی پورت ۹۲۰۰ اجرا کنید و سپس ادامه دهید.
</div>

In [4]:
print(elasticsearch.__version__)
es = Elasticsearch("http://localhost:9200")

(8, 3, 1)


<div dir="rtl">
با استفاده از دستور زیر نیز چک میکنیم که سرویس elasticsearch در حال اجرا است.
</div>

In [5]:
es.ping()

True

<div dir="rtl">
<span style="color:red"><h3>تبدیل داده های قرآنی به Dataframe و سپس تبدیل آن به داده های مناسب برای elasticsearch</h3></span>
در این مرحله نیاز داریم تا داده خامی که داریم را:
<ul>
<li>ابتدا به دیتافریم در آوریم تا مرتب شود</li>
<li>سپس آن را به فرمت json که برای ارتباط با DB های Document-based مثل همین elasticsearch است در بیاوریم</li>
<li>در نهایت با استفاده از یک generator، این json را مرتب و قابل خواندن برای elasticsearch کنیم</li>
</ul>
</div>

<div dir="rtl">
ابتدا متن قرآن به همراه آیه و سوره مربوطه اش را که از سایت <a href = "tanzil.net">تنزیل</a> دریافت کردیم، را در قالب dataframe در می آوریم
</div>

In [6]:
quran_path = './quran-simple.txt'
quran_df = pd.read_csv(quran_path,sep='|')
quran_df

,chapter,verse,text
0,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,1,4,مَالِكِ يَوْمِ الدِّينِ
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...
6231,114,2,مَلِكِ النَّاسِ
6232,114,3,إِلَـٰهِ النَّاسِ
6233,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


<div dir="rtl">
سپس dataframe امان را در قالب json در می آوریم.
</div>

In [7]:
json_from_df = quran_df.to_dict('records')
json_from_df[0]

{'chapter': 1, 'verse': 1, 'text': 'بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ'}

<div dir="rtl">
سپس با استفاده از یک تابع generator، فرمت خواسته شده توسط elasticsearch را ایجاد میکنیم تا داده آماده آپلود به DB شود.
</div>

In [8]:
def generator(df):
    for c, line in enumerate(df):
        yield{
            '_index': 'quran',
            '_id': c,
            '_source': {
                'chapter': line.get('chapter', None),
                'verse': line.get('verse', None),
                'text': line.get('text', None),
            }
        }
    # raise StopIteration

<div dir="rtl">
یک نمونه ورودی ساخته شده برای elasticsearch
</div>

In [9]:
next(generator(json_from_df))

{'_index': 'quran',
 '_id': 0,
 '_source': {'chapter': 1,
  'verse': 1,
  'text': 'بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ'}}

<div dir="rtl">
<span style="color:red"><h3>۳-اعمال settingها و mapping های مورد نظر برای تمیز کردن داده های قرآنی</h3></span>
</div>

<div dir="rtl">
 برای normalization و حذف اعراب و... در elasticsearch و از همه مهمتر، فعال کردن قابلیت analyze برای زبان عربی، نیاز داریم تا یک سری تنظیمات به elasticsearch برای index ای که ایجاد میکنیم هم بذاریم.
مهم ترین این تنظیمات، تنظیم زبان عربی برای text field است که در اصل آیات ذخیره میشوند و همچنین تنظیم فیلتر arabic_normalization است.
</div>

In [10]:
settings = {
    "settings":{
        "analysis":{
            "filter": {
                "arabic_stop": {
                "type":       "stop",
                "stopwords":  "_arabic_" 
                },
                "arabic_keywords": {
                "type":       "keyword_marker",
                "keywords":   ["مثال"] 
                },
                "arabic_stemmer": {
                "type":       "stemmer",
                "language":   "arabic"
                }
            },
            "analyzer":{
                "rebuilt_arabic":{
                    "type": "custom",
                    "tokenizer":"standard",
                    "filter":[
                    "lowercase",
                    "decimal_digit",
                    "arabic_stop",
                    "arabic_normalization",
                    ]
                }
            }
        }
    },
    "mappings":{
        "properties":{
            "text":{
                "type": "text",
                "analyzer": "arabic"
            }
        }
    }
}

<div dir="rtl">
در نهایت، indexای که قرار است تا document ها که در اصل همان آیات هستند را قرار دهیم، با تنظیماتی که اعمال کردیم، میسازیم.
</div>

In [11]:
es.indices.create(index='quran', ignore = [400, 404], body=settings)

/var/folders/7z/111t5wv14k194p7f0gnjzmg00000gn/T/ipykernel_68922/3686465265.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='quran', ignore = [400, 404], body=settings)
/var/folders/7z/111t5wv14k194p7f0gnjzmg00000gn/T/ipykernel_68922/3686465265.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='quran', ignore = [400, 404], body=settings)


ObjectApiResponse({'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [quran/fM-zxebZQh6CKMK6Sw3UdA] already exists', 'index_uuid': 'fM-zxebZQh6CKMK6Sw3UdA', 'index': 'quran'}], 'type': 'resource_already_exists_exception', 'reason': 'index [quran/fM-zxebZQh6CKMK6Sw3UdA] already exists', 'index_uuid': 'fM-zxebZQh6CKMK6Sw3UdA', 'index': 'quran'}, 'status': 400})

<div dir="rtl">
<span style="color:red"><h3>آپلود تمامی داده های قرآنی به دیتابیس elasticsearch</h3></span>
در نهایت برای index نهایی آیات، آن ها را به صورت bulk و در قالب document ها در index ای که ساختیم آپلود میکنیم
</div>

In [12]:
try:
    res = helpers.bulk(es, generator(json_from_df))
    print("Uploaded")
except Exception as e:
    print(e)

Uploaded


<div dir = "rtl">
<span style="color:red"><h3>اعمال پرسمان</h3></span>
در نهایت، با دستور `search` در `elasticsearch` روی index ای که ساختیم، پرسمان مورد نظر را اعمال میکنیم و k خروجی برتر را خروجی میدهیم.
</div>

In [13]:
res = es.search(index="quran",source=['chapter', 'verse', 'text'], size=10, query = {"match": {"text":"الناس"}})["hits"]["hits"]
a = []
for i in res:
    a.append({
        "verse": i["_source"]["text"],
        "verse_number": i["_source"]["verse"],
        "surah_number": i["_source"]["chapter"],
    })
print(a)

[{'verse': 'مَلِكِ النَّاسِ', 'verse_number': 2, 'surah_number': 114}, {'verse': 'إِلَـٰهِ النَّاسِ', 'verse_number': 3, 'surah_number': 114}, {'verse': 'إِنَّ اللَّهَ لَا يَظْلِمُ النَّاسَ شَيْئًا وَلَـٰكِنَّ النَّاسَ أَنفُسَهُمْ يَظْلِمُونَ', 'verse_number': 44, 'surah_number': 10}, {'verse': 'مِنَ الْجِنَّةِ وَالنَّاسِ', 'verse_number': 6, 'surah_number': 114}, {'verse': 'وَمَا أَرْسَلْنَاكَ إِلَّا كَافَّةً لِّلنَّاسِ بَشِيرًا وَنَذِيرًا وَلَـٰكِنَّ أَكْثَرَ النَّاسِ لَا يَعْلَمُونَ', 'verse_number': 28, 'surah_number': 34}, {'verse': 'لَخَلْقُ السَّمَاوَاتِ وَالْأَرْضِ أَكْبَرُ مِنْ خَلْقِ النَّاسِ وَلَـٰكِنَّ أَكْثَرَ النَّاسِ لَا يَعْلَمُونَ', 'verse_number': 57, 'surah_number': 40}, {'verse': 'وَلَقَدْ صَرَّفْنَا لِلنَّاسِ فِي هَـٰذَا الْقُرْآنِ مِن كُلِّ مَثَلٍ فَأَبَىٰ أَكْثَرُ النَّاسِ إِلَّا كُفُورًا', 'verse_number': 89, 'surah_number': 17}, {'verse': 'وَقِيلَ لِلنَّاسِ هَلْ أَنتُم مُّجْتَمِعُونَ', 'verse_number': 39, 'surah_number': 26}, {'verse': 'يَغْشَى النَّاسَ هَـٰذَا

<hr>

<div dir = "rtl">
<span style="color:Green"><h2>بررسی سیستم</h2></span>
۱۰ پرسمان زیر را روی سیستم اجرا میکنیم و ۱۰ خروجی برتر آن را مشاهده میکنیم.
</div>

In [14]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    with open("es_queries.txt") as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
    queries_df_columns = ["query", "rank", "result", "chapter_num", "verse_num"]
    queries_df = pd.DataFrame(columns=queries_df_columns)
    for line in lines:
        new_row = {"query": line}
        a = es.search(index="quran",source=['chapter', 'verse', 'text'], size=10, query = {"match": {"text":line}})["hits"]["hits"]
        for idx, i in enumerate(a):
            # print(i["_source"])
            new_row["rank"] = idx + 1
            new_row["result"] = i["_source"]["text"]
            new_row["chapter_num"] = i["_source"]["chapter"]
            new_row["verse_num"] = i["_source"]["verse"]
            queries_df = queries_df.append(new_row, ignore_index=True) 
queries_df

,query,rank,result,chapter_num,verse_num
0,الحمد لله,1,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,1,2
1,الحمد لله,2,وَالْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,37,182
2,الحمد لله,3,فَقُطِعَ دَابِرُ الْقَوْمِ الَّذِينَ ظَلَمُوا ...,6,45
3,الحمد لله,4,وَقُلِ الْحَمْدُ لِلَّهِ سَيُرِيكُمْ آيَاتِهِ ...,27,93
4,الحمد لله,5,وَقَالُوا الْحَمْدُ لِلَّهِ الَّذِي أَذْهَبَ ع...,35,34
...,...,...,...,...,...
75,امر معروف نهی منکر,6,فَذَاقَتْ وَبَالَ أَمْرِهَا وَكَانَ عَاقِبَةُ ...,65,9
76,امر معروف نهی منکر,7,إِلَىٰ فِرْعَوْنَ وَمَلَئِهِ فَاتَّبَعُوا أَمْ...,11,97
77,امر معروف نهی منکر,8,وَيَسِّرْ لِي أَمْرِي,20,26
78,امر معروف نهی منکر,9,وَأَشْرِكْهُ فِي أَمْرِي,20,32


<div dir="rtl">
در این نهایت، این نتایج را در فایل `es_search_res.csv` ذخیره میکنیم.
</div>

In [15]:
queries_df.to_csv("es_search_res.csv")

<div dir = "rtl">
<span style="color:Red"><h3>تحلیل نتایج</h3></span>
برای تحلیل نتایج، صرفا ۳ نتیجه برتر هر پرسمان را در یک Dataframe ذخیره میکنیم و در نهایت، معیار MRR را برای آن ذخیره میکنیم.
</div>

In [34]:
pd.set_option('display.max_colwidth', None)
top_res = queries_df.loc[queries_df['rank'].isin([1,2,3])]
# top_res = top_res.style.set_caption("نتایج برتر جستجو ۱۰ پرسمان برای elasticsearch")
top_res

,query,rank,result,chapter_num,verse_num
0,الحمد لله,1,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,1,2
1,الحمد لله,2,وَالْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,37,182
2,الحمد لله,3,فَقُطِعَ دَابِرُ الْقَوْمِ الَّذِينَ ظَلَمُوا وَالْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,6,45
10,فرعون موسی,1,إِلَىٰ فِرْعَوْنَ وَمَلَئِهِ فَاتَّبَعُوا أَمْرَ فِرْعَوْنَ وَمَا أَمْرُ فِرْعَوْنَ بِرَشِيدٍ,11,97
11,فرعون موسی,2,فِرْعَوْنَ وَثَمُودَ,85,18
12,فرعون موسی,3,وَفِرْعَوْنَ ذِي الْأَوْتَادِ,89,10
20,رب العالمين,1,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,1,2
21,رب العالمين,2,وَإِنَّهُ لَتَنزِيلُ رَبِّ الْعَالَمِينَ,26,192
22,رب العالمين,3,وَالْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,37,182
30,الصراط المستقيم,1,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ,1,6


<div dir="rtl">
همانطور که مشاهده میشود، تمامی نتایج اول، مرتبط هستند، پس معیار MRR را میتوان ۱ در نظر گرفت که این نتیجه بسیار خوبی است!
</div>

<div dir = "rtl">
<span style="color:Red"><h3>مقایسه نتایج با دیگر سیستم ها</h3></span>
خروجی سیستم های transformer، بولین، fasttext و tf-idf درآورده شده اند و برای مقایسه نتایج، خروجی برتر هر سیستم را برای هر پرسمان نشان میدهیم و سپس این خروجی را تحلیل میکنیم.
</div>

In [35]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    queries_df_columns = ["Query","Model Name", "Top Result"]
    new_models_df = pd.DataFrame(columns=queries_df_columns)
    #reading queries
    with open("es_queries.txt") as file:
        lines_queries = file.readlines()
        lines_queries = [line.rstrip() for line in lines_queries]
    #reading file names that stored other model answers
    with open("file_names.txt") as file:
        lines_names = file.readlines()
        lines_names = [line.rstrip() for line in lines_names]
    for name in lines_names:
        file_name = name.split(";")[0]
        model_name = name.split(";")[1]
        q_df = pd.read_csv(file_name)
        for query in lines_queries:
            found_q_idx = q_df[q_df['Query'].str.contains(query,na=False)].index.values.astype(int)[0] + 1
            res = q_df.iloc[[found_q_idx]]['Out'].tolist()
            next_row = {}
            next_row["Query"] = query
            next_row["Model Name"] = model_name
            next_row["Top Result"] = res[0]
            new_models_df = new_models_df.append(next_row, ignore_index=True) 
    new_top_res = top_res.loc[top_res["rank"] == 1]
    new_top_res.drop(['rank', 'chapter_num', 'verse_num'], axis=1, inplace=True)
    new_top_res.rename(columns = {'query':'Query', 'result':'Top Result'}, inplace = True)
    new_top_res["Model Name"] = "Elasticsearch"
    new_models_df = new_models_df.append(new_top_res)
new_models_df

,Query,Model Name,Top Result
0,الحمد لله,Transformer,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
1,فرعون موسی,Transformer,فِرْعَوْنَ وَ ثَمُودَ
2,رب العالمين,Transformer,فَمَا ظَنُّكُمْ بِرَبِّ الْعَالَمِينَ
3,الصراط المستقيم,Transformer,اهْدِنَا الصِّرَاطَ الْمُسْتَقِيمَ
4,ارض شمس,Transformer,الشَّمْسُ وَ الْقَمَرُ بِحُسْبَانٍ
5,نبی رسول,Transformer,وَ كَمْ أَرْسَلْنَا مِنْ نَبِيٍّ فِي الْأَوَّلِينَ
6,شیطان رجیم,Transformer,وَ مَا هُوَ بِقَوْلِ شَيْطَانٍ رَجِيمٍ
7,جنات تجری الانهار,Transformer,جَنَّاتُ عَدْنٍ تَجْرِي مِنْ تَحْتِهَا الْأَنْهَارُ خَالِدِينَ فِيهَا وَ ذَلِكَ جَزَاءُ مَنْ تَزَكَّىٰ
8,یوم القیامه,Transformer,يَسْأَلُ أَيَّانَ يَوْمُ الْقِيَامَةِ
9,امر معروف نهی منکر,Transformer,كَانُوا لَا يَتَنَاهَوْنَ عَنْ مُنْكَرٍ فَعَلُوهُ لَبِئْسَ مَا كَانُوا يَفْعَلُونَ


<div dir = "rtl">
همانطور که مشاهده میشود، با توجه به دیتاست کوچکی که داشتیم، پاسخ های tf-idf و elasticsearch از باقی روش ها بهتر عمل کرده اند و در مجموع، elasticsearch از همه روش ها بهتر عمل کرده است.
</div>